In [1]:
from resources.helper.helper_dataset import get_dataloaders_mnist
from resources.helper.helper_evaluation import set_all_seeds, set_deterministic
from resources.helper.helper_train import train_model
from resources.helper.helper_plotting import plot_training_loss, plot_accuracy, show_examples

import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
train_loader, valid_loader, test_loader = get_dataloaders_mnist(batch_size=256,validation_fraction=0.1)
for images, labels in train_loader:  
    print('Image batch d    imensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Image batch d    imensions: torch.Size([256, 1, 28, 28])
Image label dimensions: torch.Size([256])
Class labels of 10 examples: tensor([4, 0, 2, 6, 1, 3, 3, 1, 7, 2])


In [4]:
class MLP(torch.nn.Module):

    def __init__(self, num_features, num_classes, drop_proba, 
                 num_hidden_1, num_hidden_2):
        super().__init__()
        
        self.network = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(num_features, num_hidden_1,bias=False),
            torch.nn.BatchNorm1d(num_hidden_1),
            torch.nn.ReLU(),
            torch.nn.Linear(num_hidden_1, num_hidden_2,bias=False),
            torch.nn.BatchNorm1d(num_hidden_2),
            torch.nn.ReLU(),
            torch.nn.Dropout(drop_proba),

            torch.nn.Linear(num_hidden_2, num_classes)
        )

        # Kaiming He weight initialization
        # for m in self.modules():
        #     if isinstance(m, torch.nn.Linear):
        #         torch.nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
        #         if m.bias is not None:
        #             m.bias.detach().zero_()
           
        # normal weight initialization (gaussian distribution)
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                m.weight.detach().normal_(0, 0.001)
                if m.bias is not None:
                    m.bias.detach().zero_()
    def forward(self, x):
        logits = self.network(x)
        return logits

In [ ]:
torch.manual_seed(42)
model = MLP(num_features=28*28,
                             num_hidden_1=70,
                             num_hidden_2=50,
                             drop_proba=0.2,
                             num_classes=10)
model = model.to(device)
lr=0.1
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
Epochs=5
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, mode='max')

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=Epochs,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=device,
    logging_interval=50,
    scheduler=scheduler,
    scheduler_on='valid_acc')

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=Epochs,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=20)

plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)

plt.ylim([80, 100])
plt.show()


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'